# FreeMap
> ```map``` function on free json like structure

In [ ]:
# default_exp freemap

In [1]:
# export
from collections import OrderedDict

In [123]:
# export
class FreeMap:
    def __init__(self,
                 func= lambda x:x,
                 filter_function = lambda x: True,
                 flatten = False):
        """
        run map operation on a free stucture, 
        like dictionary, list within dictionary, etc
        
        """
        self.func = func
        self.filter_function = filter_function
        self.flatten = flatten
        if flatten:
            self.result_list = []
        self.create_doc()
        
    def __repr__(self):
        return f"""<A map function for free structure>
        Please put in one of the following type: 
        dict, list, tuple, set, OrderedDict
        
        {self.keep_structure}
        ======
        Mapped function
        {self.func}
        Value Filter function
        {self.filter_function}
        """
    
    @property
    def keep_structure(self):
        return "This operation will flatten the structure to a list"\
            if self.flatten else\
            "This operation will keep the original structure"
            
    def create_doc(self):
        doc = f"""
        map function for list,dict,tuple,set,OrderedDict
        {self.keep_structure}
        
        """
        if hasattr(self.func,"__doc__"):
            if self.func.__doc__!=None:
                doc += f"doc string from mapping function:\n\t\t{self.func.__doc__}\t"
        setattr(self,"__doc__",doc)
        
    def parse(self,thing):
        if type(thing) in [list,tuple,set]:
            return self.parse_seq(thing)
        elif type(thing) in [dict,OrderedDict]:
            return self.parse_kv(thing)
        else:
            return self.parse_value(thing)
        
    def parse_seq(self,thing):
        return type(thing)(self.parse(i) for i in thing)
    
    def parse_kv(self,thing):
        return type(thing)((k,self.parse(v)) for k,v in thing.items())
    
    def parse_value(self,thing):
        if self.filter_function(thing):
            if self.flatten==False:
                return self.func(thing)
            else:
                self.result_list.append(self.func(thing))
        return thing
        
    def __call__(self,iterable):
        result = self.parse(iterable)
        if self.flatten:
            return self.result_list
        else:
            return result

## Examples

In [100]:
from math import floor

In [101]:
examp = {"somekey":"somestring",
         "somekey_with_heavy_value":[1,
                                     "2",
                                     "word",
                                     {"function":floor},
                                     {"are",
                                      "is",
                                      "is",
                                      ("Lovely",
                                       "example",)}],
         "someanother_key":"hello this is a long string with verbosity"}

In [102]:
capitalize = FreeMap(lambda x:x.upper(),lambda x:(type(x)==str))
capitalize

<A map function for free structure>
        Please put in one of the following type: 
        dict, list, tuple, set, OrderedDict
        
        This operation will keep the original structure
        Mapped function
        <function <lambda> at 0x10fc0fe60>
        Value Filter function
        <function <lambda> at 0x10fc0fef0>
        

In [103]:
capitalize(examp)

{'somekey': 'SOMESTRING',
 'somekey_with_heavy_value': [1,
  '2',
  'WORD',
  {'function': <function math.floor(x, /)>},
  {('LOVELY', 'EXAMPLE'), 'ARE', 'IS'}],
 'someanother_key': 'HELLO THIS IS A LONG STRING WITH VERBOSITY'}

## Flatten and map

In [121]:
FreeMap(lambda x:x.upper(),lambda x:(type(x)==str),flatten=True)(examp)

['SOMESTRING',
 '2',
 'WORD',
 'IS',
 'LOVELY',
 'EXAMPLE',
 'ARE',
 'HELLO THIS IS A LONG STRING WITH VERBOSITY']

Just simple flatten, the default map function doesn't do anything

In [124]:
FreeMap(flatten=True)(examp)

['somestring',
 1,
 '2',
 'word',
 <function math.floor(x, /)>,
 'is',
 'Lovely',
 'example',
 'are',
 'hello this is a long string with verbosity']